In [ ]:
from pathlib import Path
import numpy as np
from funcionesTP import *

def cargarDataset(carpeta: Path):
    pathCats = carpeta.joinpath('cats/efficientnet_b3_embeddings.npy')
    pathDogs = carpeta.joinpath('dogs/efficientnet_b3_embeddings.npy')

    embeddingsCats = np.load(pathCats)
    embeddingsDogs = np.load(pathDogs)

    embeddings = np.concatenate((embeddingsCats, embeddingsDogs), axis=1)
    _, m = embeddings.shape

    Y = np.zeros((2,m))
    for i in range(0, int(m/2)):
        Y[0,i] = 1

    for i in range(int(m/2), m):
        Y[1,i] = 1

    return embeddings, Y

def cargarDatasetCompleto():
    Xt, Yt = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/train'))
    Xv, Yv = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))

    return Xt, Yt, Xv, Yv



In [8]:
Xt, Yt, Xv, Yv = cargarDatasetCompleto()

def test_W(W):
    Xv, Yv = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))

    matrizDiferencia = Yv - (W @ Xv)
    print(np.linalg.norm(matrizDiferencia, ord=2))



In [ ]:
def generar_W_EcuacionesNormales():

    W = pinvEcuacionesNormales(Xt, Yt)
    np.save('./W_eqnormales.npy', W)
    
def testEqNormales():
    W = np.load('./W_eqnormales.npy')
    test_W(W)

generar_W_EcuacionesNormales()
testEqNormales()

producto matricial


  1%|          | 15/2000 [00:17<39:12,  1.19s/it]

In [ ]:
def generar_W_QRHH():

    Q, R = QR_con_HH(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrhh.npy', W)

def testQRHH():
    W = np.load('./W_qrhh.npy')
    test_W(W)

generar_W_QRHH()
testQRHH()

In [ ]:
def generar_W_QRGS():

    Q, R = QR_con_GS(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrgs', W)

def testQRGS():
    W = np.load('./W_qrgs.npy')
    test_W(W)
    
generar_W_QRGS()
testQRGS()

In [ ]:
def generar_W_SVD():
    W = svdFCN(Xt, Yt)
    np.save('./W_SVD.npy', W)

def testSVD():
    W = np.load('./W_SVD.npy')
    test_W(W)
    
generar_W_SVD()
testSVD()